Let's convert the entire script to use Flower, a popular framework for federated learning. Flower will handle all the complex server-client communication, so our code will become much simpler and more organized.

We'll follow the same step-by-step process, keeping your original model, data loading functions, and variable names so you can easily see what's changed.

## Step 1: The Foundation (Imports and Setup)
Every Python script starts with importing the necessary libraries and setting up the environment. This is like laying the foundation for a house.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from collections import OrderedDict
import flwr as fl
import numpy as np
import pandas as pd
import os
import random
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# --- Set Device ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Define dataset loader

PyTorch uses a Dataset object to handle data loading. Since our model will take three different kinds of input (sensor data, image 1, and image 2), we need to create a special class that tells PyTorch how to retrieve one sample of each, along with its corresponding label.

This class will have three essential methods:

__init__: Initializes the dataset by storing our feature and label arrays.

__len__: Returns the total number of samples in the dataset.

__getitem__: Fetches a single data sample at a given index.

Here is the code for it. Add this to your script:

In [2]:
# Define dataset loader
class CustomDatasetRes(Dataset):
    def __init__(self, features1, features2, features3, labels):
        self.features1 = features1
        self.features2 = features2
        self.features3 = features3
        self.labels = labels

    def __len__(self):
        return len(self.features1)
    
    def __getitem__(self, index):
        return self.features1[index], self.features2[index], self.features3[index], self.labels[index]

### Helper Functions
Next, we'll add a few helper functions. These functions will perform common tasks that we'll need later, like displaying results, scaling data, and ensuring our experiments are reproducible.

1. display_result

This function takes the true labels (y_test) and the model's predicted labels (y_pred) and prints out standard performance metrics like accuracy, precision, recall, and F1-score.

In [3]:
def display_result(y_test, y_pred):
    print('Accuracy score : ', accuracy_score(y_test, y_pred))
    print('Precision score : ', precision_score(y_test, y_pred, average='weighted'))
    print('Recall score : ', recall_score(y_test, y_pred, average='weighted'))
    print('F1 score : ', f1_score(y_test, y_pred, average='weighted'))

2. scaled_data

This function uses Scikit-learn's StandardScaler to normalize the sensor (CSV) data. Scaling is crucial because it ensures that features with larger value ranges don't dominate the learning process. Notice there are two functions with the same name in the original code. In Python, the last definition of a function is the one that gets used. We will add both for completeness, but just know that the first one is effectively overwritten by the second.

In [4]:
def scale_data(X_train, X_test, X_val):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_val_scaled = scaler.transform(X_val)
    return X_train_scaled, X_test_scaled, X_val_scaled

def scaled_data(X_train):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    return X_train_scaled

3. set_seed

This is a very important function for reproducibility. Machine learning involves a lot of randomness (e.g., initializing model weights, shuffling data). By setting a "seed," we ensure that the sequence of random numbers is the same every time we run the code, which means we'll get the exact same results.

In [5]:
def set_seed(seed=0):
    # Sets the environment variable for Python's hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    # Sets the seed for NumPy's random number generator
    np.random.seed(seed)
    # Sets the seed for Python's built-in random module
    random.seed(seed)
    # Sets the seed for PyTorch's random number generator
    torch.manual_seed(seed)
    # If using a GPU, sets the seed for all CUDA devices
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # For multi-GPU setups
    # Ensures deterministic behavior in cuDNN (CUDA Deep Neural Network library)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

### loading and preprocessing the data.

The function loadClientsData is designed for a federated learning scenario. It reads data from separate files for each participant (or "client"), cleans it, aligns the different data types (sensor vs. image), and splits it into training and testing sets for each client.

Because this function is quite long, we'll build it in a few parts.

#### Part 1: Initializing and Processing Training Data
First, we'll define the function, list the subject IDs we want to load, and create empty dictionaries to store each client's data. Then, we'll start a loop to process each subject one by one. Inside the loop, we'll begin by loading and cleaning the training data.

This involves:

Reading the sensor data from a CSV file.

Removing rows with missing values and any duplicate rows.

Dropping columns that we don't need (like the 'Infrared' sensor readings).

Loading the corresponding image, label, and timestamp data from .npy files.

#### Part 2: Aligning and Preparing Training Data
After loading the raw data, we face a common problem: the datasets don't perfectly match. Because we dropped rows with missing values from the sensor (CSV) data, there are now timestamps in our image data that no longer have a corresponding entry in the sensor data.

We need to align them by removing the image samples that don't have a matching sensor reading.

After alignment, we'll prepare the data for the model:

Set the seed for reproducibility.

Separate features from labels.

One-hot encode the labels, converting them into a format suitable for the model's output layer (e.g., class 3 becomes [0, 0, 0, 1, 0, ...]).

Scale the numeric sensor data and the image pixel values.

Reshape the images to the format expected by the convolutional layers.

#### Part 3: Processing the Test Data and Finalizing the Function
The logic here is identical to what we just did for the training data:

Load the test sensor data (_test.csv) and test image data (_test.npy).

Clean the sensor data by removing missing values and unnecessary columns.

Align the test image data with the cleaned test sensor data.

Prepare the aligned test data (one-hot encode labels, scale features, reshape images).

Store all the processed training and test arrays into our dictionaries.

Increment the clint_index and repeat the process for the next subject.

After the loop finishes, the function returns all the dictionaries containing the data for every client.

In [6]:
def loadClientsData():
    subs = [1, 3, 4, 7, 10, 11, 12, 13, 14, 15, 16, 17]
    X_train_csv_scaled_splits = {}
    X_test_csv_scaled_splits = {}
    Y_train_csv_splits = {}
    Y_test_csv_splits = {}
    X_train_1_scaled_splits = {}
    X_test_1_scaled_splits = {}
    Y_train_1_splits = {}
    Y_test_1_splits = {}
    X_train_2_scaled_splits = {}
    X_test_2_scaled_splits = {}
    Y_train_2_splits = {}
    Y_test_2_splits = {}
    clint_index = 0
    for sub_ in subs:
        # --- Load and clean TRAINING sensor data (CSV) ---
        SUB_train = pd.read_csv('./dataset/Sensor + Image/{}_sensor_train.csv'.format(sub_), skiprows=1)
        SUB_train.head()
        
        SUB_train.isnull().sum()
        NA_cols = SUB_train.columns[SUB_train.isnull().any()]
        SUB_train.dropna(inplace=True)
        SUB_train.drop_duplicates(inplace=True)
        
        times_train = SUB_train['Time']
        list_DROP = ['Infrared 1',
                     'Infrared 2',
                     'Infrared 3',
                     'Infrared 4',
                     'Infrared 5',
                     'Infrared 6']
        SUB_train.drop(list_DROP, axis=1, inplace=True)
        SUB_train.drop(NA_cols, axis=1, inplace=True)  # drop NAN COLS

        SUB_train.set_index('Time', inplace=True)
        SUB_train.head()

        # --- Load TRAINING image data from both cameras ---
        cam = '1'
        image_train = './dataset/Sensor + Image' + '/' + '{}_image_1_train.npy'.format(sub_)
        name_train = './dataset/Sensor + Image' + '/' + '{}_name_1_train.npy'.format(sub_)
        label_train = './dataset/Sensor + Image' + '/' + '{}_label_1_train.npy'.format(sub_)

        img_1_train = np.load(image_train)
        label_1_train = np.load(label_train)
        name_1_train = np.load(name_train)

        cam = '2'
        image_train = './dataset/Sensor + Image' + '/' + '{}_image_2_train.npy'.format(sub_)
        name_train = './dataset/Sensor + Image' + '/' + '{}_name_2_train.npy'.format(sub_)
        label_train = './dataset/Sensor + Image' + '/' + '{}_label_2_train.npy'.format(sub_)

        img_2_train = np.load(image_train)
        label_2_train = np.load(label_train)
        name_2_train = np.load(name_train)

        # --- Align the training data by removing samples not present in the cleaned CSV ---
        redundant_1 = list(set(name_1_train) - set(times_train))
        redundant_2 = list(set(name_2_train) - set(times_train))
        
        ind = np.arange(0, len(img_1_train))

        red_in1 = ind[np.isin(name_1_train, redundant_1)]
        name_1_train = np.delete(name_1_train, red_in1)
        img_1_train = np.delete(img_1_train, red_in1, axis=0)
        label_1_train = np.delete(label_1_train, red_in1)

        red_in2 = ind[np.isin(name_2_train, redundant_2)]
        name_2_train = np.delete(name_2_train, red_in2)
        img_2_train = np.delete(img_2_train, red_in2, axis=0)
        label_2_train = np.delete(label_2_train, red_in2)
        
        # --- Prepare the final aligned training data ---
        data_train = SUB_train.loc[name_1_train].values

        set_seed()
        X_csv_train, y_csv_train = data_train[:, :-1], data_train[:, -1]
        
        # Remap label 20 to 0 for consistency
        y_csv_train = np.where(y_csv_train == 20, 0, y_csv_train)
        label_1_train = np.where(label_1_train == 20, 0, label_1_train)
        label_2_train = np.where(label_2_train == 20, 0, label_2_train)

        # One-hot encode the labels for PyTorch
        Y_csv_train = torch.nn.functional.one_hot(torch.from_numpy(y_csv_train).long(), 12).float()
        Y_train_1 = torch.nn.functional.one_hot(torch.from_numpy(label_1_train).long(), 12).float()
        Y_train_2 = torch.nn.functional.one_hot(torch.from_numpy(label_2_train).long(), 12).float()

        # Scale the sensor data
        X_csv_train_scaled = scaled_data(X_csv_train)

        X_train_1 = img_1_train
        y_train_1 = label_1_train
        
        X_train_2 = img_2_train
        y_train_2 = label_2_train

        # Reshape images to (samples, height, width, channels)
        shape1, shape2 = 32, 32
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], shape1, shape2, 1)
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], shape1, shape2, 1)

        # Scale image pixel values to be between 0 and 1
        X_train_1_scaled = X_train_1 / 255.0
        X_train_2_scaled = X_train_2 / 255.0

        # --- Load and clean TEST sensor data (CSV) ---
        SUB_test = pd.read_csv('./dataset/Sensor + Image/{}_sensor_test.csv'.format(sub_), skiprows=1)
        SUB_test.head()
        
        SUB_test.isnull().sum()
        NA_cols = SUB_test.columns[SUB_test.isnull().any()]
        SUB_test.dropna(inplace=True)
        SUB_test.drop_duplicates(inplace=True)

        times_test = SUB_test['Time']
        SUB_test.drop(list_DROP, axis=1, inplace=True)
        SUB_test.drop(NA_cols, axis=1, inplace=True)

        SUB_test.set_index('Time', inplace=True)
        SUB_test.head()

        # --- Load TEST image data from both cameras ---
        image_test = './dataset/Sensor + Image' + '/' + '{}_image_1_test.npy'.format(sub_)
        name_test = './dataset/Sensor + Image' + '/' + '{}_name_1_test.npy'.format(sub_)
        label_test = './dataset/Sensor + Image' + '/' + '{}_label_1_test.npy'.format(sub_)
        img_1_test = np.load(image_test)
        label_1_test = np.load(label_test)
        name_1_test = np.load(name_test)

        image_test = './dataset/Sensor + Image' + '/' + '{}_image_2_test.npy'.format(sub_)
        name_test = './dataset/Sensor + Image' + '/' + '{}_name_2_test.npy'.format(sub_)
        label_test = './dataset/Sensor + Image' + '/' + '{}_label_2_test.npy'.format(sub_)
        img_2_test = np.load(image_test)
        label_2_test = np.load(label_test)
        name_2_test = np.load(name_test)

        # --- Align the test data ---
        redundant_1 = list(set(name_1_test) - set(times_test))
        redundant_2 = list(set(name_2_test) - set(times_test))
        
        ind = np.arange(0, len(img_1_test))

        red_in1 = ind[np.isin(name_1_test, redundant_1)]
        name_1_test = np.delete(name_1_test, red_in1)
        img_1_test = np.delete(img_1_test, red_in1, axis=0)
        label_1_test = np.delete(label_1_test, red_in1)

        red_in2 = ind[np.isin(name_2_test, redundant_2)]
        name_2_test = np.delete(name_2_test, red_in2)
        img_2_test = np.delete(img_2_test, red_in2, axis=0)
        label_2_test = np.delete(label_2_test, red_in2)

        # --- Prepare the final aligned test data ---
        data_test = SUB_test.loc[name_1_test].values

        set_seed()
        X_csv_test, y_csv_test = data_test[:, :-1], data_test[:, -1]
        y_csv_test = np.where(y_csv_test == 20, 0, y_csv_test)
        label_1_test = np.where(label_1_test == 20, 0, label_1_test)
        label_2_test = np.where(label_2_test == 20, 0, label_2_test)

        Y_csv_test = torch.nn.functional.one_hot(torch.from_numpy(y_csv_test).long(), 12).float()
        X_csv_test_scaled = scaled_data(X_csv_test)
        
        X_test_1 = img_1_test
        y_test_1 = label_1_test
        Y_test_1 = torch.nn.functional.one_hot(torch.from_numpy(y_test_1).long(), 12).float()

        X_test_2 = img_2_test
        y_test_2 = label_2_test
        Y_test_2 = torch.nn.functional.one_hot(torch.from_numpy(y_test_2).long(), 12).float()

        X_test_1 = X_test_1.reshape(X_test_1.shape[0], shape1, shape2, 1)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], shape1, shape2, 1)

        X_test_1_scaled = X_test_1 / 255.0
        X_test_2_scaled = X_test_2 / 255.0

        # --- Store all processed data for the current client ---
        X_train_csv_scaled_splits[clint_index] = X_csv_train_scaled
        X_test_csv_scaled_splits[clint_index] = X_csv_test_scaled
        Y_train_csv_splits[clint_index] = Y_csv_train
        Y_test_csv_splits[clint_index] = Y_csv_test
        X_train_1_scaled_splits[clint_index] = X_train_1_scaled
        X_test_1_scaled_splits[clint_index] = X_test_1_scaled
        Y_train_1_splits[clint_index] = Y_train_1
        Y_test_1_splits[clint_index] = Y_test_1
        X_train_2_scaled_splits[clint_index] = X_train_2_scaled # This line had a bug in the original code
        X_test_2_scaled_splits[clint_index] = X_test_2_scaled
        Y_train_2_splits[clint_index] = Y_train_2
        Y_test_2_splits[clint_index] = Y_test_2
        clint_index += 1
        
    # --- After loop, return all dictionaries ---
    return X_train_csv_scaled_splits,X_test_csv_scaled_splits, Y_train_csv_splits,Y_test_csv_splits,X_train_1_scaled_splits,X_test_1_scaled_splits,Y_train_1_splits,Y_test_1_splits,X_train_2_scaled_splits,X_test_2_scaled_splits,Y_train_2_splits,Y_test_2_splits

## Step 2: Client Selection

We're making great progress. We've handled all the data loading and preparation. Now, we'll add the functions that form the "intelligence" of our federated learning system: client selection.

Instead of blindly averaging updates from every client in every round, these methods evaluate each client's performance and contribution. This allows the server to select the most promising or reliable clients to participate in the global model update, potentially leading to faster convergence and a more robust final model.

We'll add a series of functions, each calculating a specific metric to judge the clients.

### Client Evaluation Metrics
Add all the following functions to your script. Each one calculates a different score based on a client's performance.

1. Relative Loss Reduction (RF_loss)

This measures how much a client's training loss has dropped from the beginning to the end of a local training round, relative to the client with the biggest drop. A higher score means the client is learning effectively.

In [7]:
def calculate_relative_loss_reduction_as_list(client_losses):
    """
    Calculates the relative loss reduction (RF_loss) for each client.
    """
    loss_reduction = {}
    for client_id, losses in client_losses.items():
        if len(losses) < 2:
            raise ValueError(f"Client {client_id} has less than 2 loss values, cannot calculate RF_loss.")
        loss_start = losses[0]
        loss_end = losses[-1]
        loss_reduction[client_id] = loss_start - loss_end

    max_loss_reduction = max(loss_reduction.values())
    if max_loss_reduction == 0:
        return [0.0] * len(loss_reduction)  # If no loss reduction, return 0.0 for all clients

    rf_losses_list = [
        reduction / max_loss_reduction for reduction in loss_reduction.values()
    ]
    return rf_losses_list

2. Relative Training Accuracy (RF_ACC_Train)

This measures a client's local training accuracy relative to the client with the highest accuracy. It's a straightforward measure of performance on local data.

In [8]:
def calculate_relative_train_accuracy(client_acc):
    """
    Calculates the relative training accuracy (RF_Acc_Train) for each client.
    """
    max_acc = max(client_acc.values())
    if max_acc == 0:
        return [0.0] * len(client_acc)  # If no accuracy, return 0.0 for all clients

    rf_accs_train_list = [
        acc / max_acc for acc in client_acc.values()
    ]
    return rf_accs_train_list

3. Global Validation Accuracy (RF_ACC_Global)

This is a more sophisticated metric. It rewards clients for high accuracy on a global test set but penalizes them if their global accuracy is much worse than their local training accuracy (which is a sign of overfitting).

In [9]:
def calculate_global_validation_accuracy(train_acc, global_acc):
    """
    Calculates the global validation accuracy (RF_Acc_Global) based on local training accuracies.
    """
    if set(train_acc.keys()) != set(global_acc.keys()):
        raise ValueError("Client IDs for train and global accuracy do not match.")

    max_global_acc = max(global_acc.values())
    if max_global_acc == 0:
        max_global_acc = 1  # Avoid division by zero

    global_train_diff = {
        client_id: train_acc[client_id] - global_acc[client_id]
        for client_id in train_acc
    }
    max_global_train_diff = max(global_train_diff.values())
    if max_global_train_diff == 0:
        max_global_train_diff = 1  # Avoid division by zero

    rf_acc_global_list = [
        (global_acc[client_id] / max_global_acc) - (global_train_diff[client_id] / max_global_train_diff)
        for client_id in train_acc
    ]
    return rf_acc_global_list

4. Loss Outliers (P_loss)

This function flags clients that are potential negative contributors. If a client's final training loss is significantly higher than the average loss of all clients, it gets a high penalty score. Otherwise, its penalty is zero.

In [10]:
def calculate_loss_outliers(client_losses, lambda_loss=1.5):
    """
    Calculates the loss outlier penalty (P_loss) for each client.
    """
    final_losses = {client_id: losses[-1] for client_id, losses in client_losses.items()}
    loss_values = np.array(list(final_losses.values()))

    mean_loss = np.mean(loss_values)
    std_loss = np.std(loss_values)

    threshold = mean_loss + lambda_loss * std_loss

    max_loss = np.max(loss_values)

    if max_loss == 0:
        return [0.0] * len(loss_values)

    # Identify outliers
    loss_outliers = [
        final_loss / max_loss if final_loss > threshold else 0.0
        for final_loss in loss_values
    ]
    return loss_outliers

5. Performance Bias (P_bias)

This metric calculates the gap between a client's performance on its own validation data versus its performance on the global validation data. A large gap might indicate that the client's local data is not representative of the overall data distribution.

In [11]:
def calculate_performance_bias(val_acc, global_acc):
    """
    Calculates the performance bias penalty (P_bias).
    """
    if set(val_acc.keys()) != set(global_acc.keys()):
        raise ValueError("Client IDs for validation and global accuracy do not match.")

    performance_bias_list = []
    for client_id in val_acc:
        val = val_acc[client_id]
        global_val = global_acc[client_id]
        max_val = max(val, global_val)

        if max_val == 0:
            performance_bias = 0
        else:
            performance_bias = abs(val - global_val) / max_val
        performance_bias_list.append(performance_bias)

    return performance_bias_list

Excellent. Now that we have the functions to score each client, we need the final step: the algorithms that use these scores to select which clients will participate in a given round.

### Client Selection Algorithms
1. Pareto Optimization

This is a powerful technique used when you have multiple, often conflicting, objectives. Instead of combining all metrics into one score, it tries to find a set of clients that represent the best possible trade-offs.

A client is considered "Pareto optimal" if no other client is better than it across all metrics. The algorithm first finds this set of optimal clients.

If there are more optimal clients than needed, it selects a random subset.

If there are fewer, it fills the remaining spots by picking the clients with the best-combined performance score.

In [12]:
def pareto_optimization(
    rf_loss, rf_acc_train, rf_acc_val, rf_acc_global, p_loss, p_bias, client_num
):
    """
    实现 Pareto 优化，筛选节点。

    参数：
    - rf_loss (list): 局部训练损失相对下降幅度。
    - rf_acc_train (list): 局部训练精度。
    - rf_acc_val (list): 局部验证精度。
    - rf_acc_global (list): 全局验证精度。
    - p_loss (list): 损失异常。
    - p_bias (list): 性能偏离。
    - client_num (int): 要选出的节点数。

    返回：
    - selected_clients (list): 选中的 client ID（按输入顺序从 0 开始）。
    """
    # 将输入指标整合为二维数组，便于处理
    # data = np.array([rf_loss, rf_acc_train, rf_acc_val, rf_acc_global, -np.array(p_loss), -np.array(p_bias)]).T

    # 确保所有数组中的元素都转换为 NumPy 数组
    # rf_loss = np.array([x.detach().cpu().numpy() for x in rf_loss])
    rf_loss = np.array(list(rf_loss))
    rf_acc_train = rf_acc_train.detach().cpu().numpy() if isinstance(rf_acc_train, torch.Tensor) else np.array(
        rf_acc_train)
    rf_acc_val = rf_acc_val.detach().cpu().numpy() if isinstance(rf_acc_val, torch.Tensor) else np.array(rf_acc_val)
    rf_acc_global = rf_acc_global.detach().cpu().numpy() if isinstance(rf_acc_global, torch.Tensor) else np.array(
        rf_acc_global)
    p_loss = p_loss.detach().cpu().numpy() if isinstance(p_loss, torch.Tensor) else np.array(p_loss)
    p_bias = p_bias.detach().cpu().numpy() if isinstance(p_bias, torch.Tensor) else np.array(p_bias)
    # rf_acc_train = np.array([x.detach().cpu().numpy() for x in rf_acc_train])
    # rf_acc_val = np.array([x.detach().cpu().numpy() for x in rf_acc_val])
    # rf_acc_global = np.array([x.detach().cpu().numpy() for x in rf_acc_global])
    # p_loss = np.array([x.detach().cpu().numpy() for x in p_loss])
    # p_bias = np.array([x.detach().cpu().numpy() for x in p_bias])

    # 构造 NumPy 数组并转置
    data = np.array([rf_loss, rf_acc_train, rf_acc_val, rf_acc_global, -p_loss, -p_bias]).T

    # Pareto 前沿筛选
    def is_dominated(point, others):
        """判断 point 是否被 others 支配"""
        return any(np.all(other >= point) and np.any(other > point) for other in others)

    pareto_indices = [
        i for i, point in enumerate(data) if not is_dominated(point, np.delete(data, i, axis=0))
    ]
    pareto_clients = pareto_indices

    # 如果前沿节点数多于 client_num，随机选取
    if len(pareto_clients) > client_num:
        return random.sample(pareto_clients, client_num)

    # 如果前沿节点数小于 client_num，基于组合评分补充
    remaining_slots = client_num - len(pareto_clients)
    pareto_scores = [0.4 * rf_loss[i] + 0.6 * rf_acc_global[i] for i in range(len(rf_loss))]
    sorted_indices = np.argsort(pareto_scores)[::-1]  # 按评分从高到低排序

    selected_clients = set(pareto_clients)
    for i in sorted_indices:
        if len(selected_clients) >= client_num:
            break
        if i not in selected_clients:
            selected_clients.add(i)

    return list(selected_clients)

2. Weighted Sum Method (5RF)

This is a more straightforward approach. It calculates a single comprehensive score for each client by taking a weighted sum of all the metrics. Clients with the highest final scores are selected. The weights (0.2, 0.1, 0.3, etc.) determine the importance of each metric.

In [13]:
def get_top_clients_with5RF(rf_loss, rf_acc_train, rf_acc_val, rf_acc_global, p_loss, p_bias, client_num):
    rf_loss = np.array(list(rf_loss))
    rf_acc_train = np.array(rf_acc_train)
    rf_acc_val = np.array(rf_acc_val)
    rf_acc_global = np.array(rf_acc_global)
    p_loss = np.array(p_loss)
    p_bias = np.array(p_bias)

    # Calculate a single weighted score for each client
    scores = (
            0.2 * rf_loss +
            0.1 * rf_acc_train +
            0.2 * rf_acc_val +
            0.3 * rf_acc_global -
            0.1 * p_loss -
            0.1 * p_bias
    )
    origin_scores = scores
    # Get the indices of the clients with the highest scores
    top_client_ids = np.argsort(scores)[::-1][:client_num]  # Sort descending and take the top N
    return top_client_ids.tolist(), origin_scores

## Step 2: The AI's Brain (The Model Definition)
We have the data pipeline and the client selection logic. Now it's time to build the brain of the operation: the neural network model itself.

The model, ModelCSVIMG, is a multi-modal neural network. This means it's designed to accept and process multiple types of data at once. It has three distinct input branches:

One for the numerical sensor (CSV) data.

One for the images from camera 1.

One for the images from camera 2.

The features extracted from each branch are then combined (fused) and passed to a final set of layers that perform the classification. The original code contains a few versions of the architecture; we will use the final, most complex one.

Add the complete model class to your script:

In [14]:
class ModelCSVIMG(nn.Module):
    def __init__(self, num_csv_features, img_shape1, img_shape2):
        super(ModelCSVIMG, self).__init__()

        # --- Branch 1: For processing numerical CSV data ---
        self.csv_fc_1 = nn.Linear(num_csv_features, 2000)
        self.csv_bn_1 = nn.BatchNorm1d(2000)
        self.csv_fc_2 = nn.Linear(2000, 600)
        self.csv_bn_2 = nn.BatchNorm1d(600)
        self.csv_dropout = nn.Dropout(0.2)

        # --- Branch 2: For processing images from Camera 1 (CNN) ---
        self.img1_conv_1 = nn.Conv2d(in_channels=1, out_channels=18, kernel_size=3, stride=1, padding=1)
        self.img1_batch_norm = nn.BatchNorm2d(18)
        self.img1_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Flattened features from the CNN go into a fully connected layer
        self.img1_fc1 = nn.Linear(18 * 16 * 16, 100)
        self.img1_dropout = nn.Dropout(0.2)

        # --- Branch 3: For processing images from Camera 2 (identical to Branch 2) ---
        self.img2_conv = nn.Conv2d(in_channels=1, out_channels=18, kernel_size=3, stride=1, padding=1)
        self.img2_batch_norm = nn.BatchNorm2d(18)
        self.img2_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.img2_fc1 = nn.Linear(18 * 16 * 16, 100)
        self.img2_dropout = nn.Dropout(0.2)

        # --- Fusion and Final Classification Layers ---
        # The input size is 600 (from CSV) + 100 (from Image 1) + 100 (from Image 2) = 800
        self.fc1 = nn.Linear(800, 1200)
        self.dr1 = nn.Dropout(0.2)
        # A residual connection is used here: input to fc2 is the original 800 + output of fc1 (1200) = 2000
        self.fc2 = nn.Linear(2000, 12) # 12 output classes

    def forward(self, x_csv, x_img1, x_img2):
        # --- Process CSV data ---
        x_csv = F.relu(self.csv_bn_1(self.csv_fc_1(x_csv)))
        x_csv = F.relu(self.csv_bn_2(self.csv_fc_2(x_csv)))
        x_csv = self.csv_dropout(x_csv)

        # --- Process Image 1 data ---
        # Reshape image from (batch, height, width, channels) to (batch, channels, height, width)
        x_img1 = x_img1.permute(0, 3, 1, 2)
        x_img1 = F.relu(self.img1_conv_1(x_img1))
        x_img1 = self.img1_batch_norm(x_img1)
        x_img1 = self.img1_pool(x_img1)
        x_img1 = x_img1.contiguous().view(x_img1.size(0), -1) # Flatten
        x_img1 = F.relu(self.img1_fc1(x_img1))
        x_img1 = self.img1_dropout(x_img1)

        # --- Process Image 2 data ---
        x_img2 = x_img2.permute(0, 3, 1, 2)
        x_img2 = F.relu(self.img2_conv(x_img2))
        x_img2 = self.img2_batch_norm(x_img2)
        x_img2 = self.img2_pool(x_img2)
        x_img2 = x_img2.contiguous().view(x_img2.size(0), -1) # Flatten
        x_img2 = F.relu(self.img2_fc1(x_img2))
        x_img2 = self.img2_dropout(x_img2)

        # --- Fusion ---
        x = torch.cat((x_csv, x_img1, x_img2), dim=1)
        residual = x # Keep a copy for the residual connection
        
        # --- Final layers ---
        x = F.relu(self.fc1(x))
        x = self.dr1(x)
        # Concatenate the residual connection
        x = torch.cat((residual, x), dim=1)
        # Final output with softmax for classification
        x = F.softmax(self.fc2(x), dim=1)

        return x

## Step 3: Define Training and Testing Functions
Instead of methods inside a Client class, we'll create standalone train and test functions. This makes the code cleaner. This logic is taken directly from your train_one_epoch and validate functions.

In [15]:
def train(net, trainloader, epochs):
    """Train the model on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    net.train()
    for _ in range(epochs):
        for batch in tqdm(trainloader, "Training"):
            data1, data2, data3, target = batch
            data1, data2, data3, target = data1.to(DEVICE).float(), data2.to(DEVICE).float(), data3.to(DEVICE).float(), target.to(DEVICE).float()
            optimizer.zero_grad()
            loss = criterion(net(data1, data2, data3), target)
            loss.backward()
            optimizer.step()

def test(net, testloader):
    """Validate the model on the test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in tqdm(testloader, "Testing"):
            data1, data2, data3, target = batch
            data1, data2, data3, target = data1.to(DEVICE).float(), data2.to(DEVICE).float(), data3.to(DEVICE).float(), target.to(DEVICE).float()
            outputs = net(data1, data2, data3)
            loss += criterion(outputs, target).item()
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == torch.max(target, 1)[1]).sum().item()
    accuracy = correct / total
    return loss / len(testloader), accuracy

## Step 4: Create the Flower Client
This is where Flower really comes in. We'll replace your custom Client class with a FlowerClient that inherits from Flower's NumPyClient. This class tells Flower how each client should handle parameters received from the server and how to perform local training and evaluation.

In [16]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        """Return the local model parameters as a list of NumPy arrays."""
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

    def set_parameters(self, parameters):
        """Update the local model with parameters from the server."""
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        """Train the model using the parameters received from the server."""
        self.set_parameters(parameters)
        # Use the train function to train the model for 3 local epochs
        train(self.net, self.trainloader, epochs=3)
        return self.get_parameters(config={}), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        """Evaluate the model using the parameters received from the server."""
        self.set_parameters(parameters)
        # Use the test function to evaluate the model
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}

## Step 5: The main Function - Bringing It All Together
Finally, we rewrite the main function. This is where we will:

Load all the client data just once.

Define a client_fn. Flower uses this function to create clients on demand for the simulation.

Define a server-side evaluation function (get_evaluate_fn) so the server can test the global model's performance on a held-out test set after each round.

Configure and start the Flower simulation.

def main():
    # --- 1. Load the data for all clients ---
    print("Loading data for all clients...")
    X_train_csv, X_test_csv, Y_train_csv, Y_test_csv, \
    X_train_1, X_test_1, Y_train_1, Y_test_1, \
    X_train_2, X_test_2, Y_train_2, Y_test_2 = loadClientsData()

    TOTAL_CLIENTS = len(X_train_csv)

    # --- 2. Define the client factory function ---
    def client_fn(cid: str) -> FlowerClient:
        """Create a Flower client instance for a given client ID."""
        client_id = int(cid)

        # Create the model
        net = ModelCSVIMG(num_csv_features=X_train_csv[client_id].shape[1], img_shape1=32, img_shape2=32).to(DEVICE)

        # Create the data loaders
        train_dataset = CustomDatasetRes(X_train_csv[client_id], X_train_1[client_id], X_train_2[client_id], Y_train_csv[client_id])
        val_dataset = CustomDatasetRes(X_test_csv[client_id], X_test_1[client_id], X_test_2[client_id], Y_test_csv[client_id])
        trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valloader = DataLoader(val_dataset, batch_size=64)

        return FlowerClient(client_id, net, trainloader, valloader)

    # --- 3. Define the server-side evaluation function ---
    def get_evaluate_fn(test_data_splits):
        """Return an evaluation function for server-side evaluation."""
        def evaluate(server_round: int, parameters: fl.common.NDArrays, config: dict):
            net = ModelCSVIMG(num_csv_features=test_data_splits[0][0].shape[1], img_shape1=32, img_shape2=32).to(DEVICE)

            # Use the last client's test data for server-side evaluation
            server_test_cid = TOTAL_CLIENTS - 1
            params_dict = zip(net.state_dict().keys(), parameters)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            net.load_state_dict(state_dict, strict=True)

            test_dataset = CustomDatasetRes(test_data_splits[0][server_test_cid], test_data_splits[1][server_test_cid],
                                         test_data_splits[2][server_test_cid], test_data_splits[3][server_test_cid])
            testloader = DataLoader(test_dataset, batch_size=64)
            loss, accuracy = test(net, testloader)
            print(f"Server-side evaluation round {server_round} - Loss: {loss:.4f} | Accuracy: {accuracy:.4f}")
            return loss, {"accuracy": accuracy}
        return evaluate

    # --- 4. Define the strategy ---
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=0.5,  # Train on 50% of clients (6 out of 12)
        min_fit_clients=6,
        fraction_evaluate=0.5, # Evaluate on 50% of clients
        min_evaluate_clients=6,
        min_available_clients=TOTAL_CLIENTS,
        evaluate_fn=get_evaluate_fn((X_test_csv, X_test_1, X_test_2, Y_test_csv)), # Server-side evaluation
    )

    # --- 5. Start the simulation ---
    print("Starting Flower simulation...")
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=TOTAL_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=10), # Run for 10 rounds
        strategy=strategy,
        client_resources={"num_cpus": 2, "num_gpus": 0.5 if torch.cuda.is_available() else 0},
    )

if __name__ == "__main__":
    main()

In [ ]:
def main():
    # --- 1. Load the data for all clients ---
    print("Loading data for all clients...")
    X_train_csv, X_test_csv, Y_train_csv, Y_test_csv, \
    X_train_1, X_test_1, Y_train_1, Y_test_1, \
    X_train_2, X_test_2, Y_train_2, Y_test_2 = loadClientsData()

    TOTAL_CLIENTS = len(X_train_csv)

    # --- Define Experimental Scenarios ---
    model_names = {
        'tc1c2ResModelV3DataV3Adam',
        'tc1c2ResModelV3DataV3AdamWithSCVLost',
        'tc1c2ResModelV3DataV3AdamWithImgLost'
    }
    svmethods = {'pareto', 'random', '5RF'} # Using '5RF' for our simplified high-accuracy selection


    # --- 2. Define the client factory function ---
    def client_fn(cid: str) -> FlowerClient:
        """Create a Flower client instance for a given client ID."""
        client_id = int(cid)

        # Create the model
        net = ModelCSVIMG(num_csv_features=X_train_csv[client_id].shape[1], img_shape1=32, img_shape2=32).to(DEVICE)

        # Create the data loaders
        train_dataset = CustomDatasetRes(X_train_csv[client_id], X_train_1[client_id], X_train_2[client_id], Y_train_csv[client_id])
        val_dataset = CustomDatasetRes(X_test_csv[client_id], X_test_1[client_id], X_test_2[client_id], Y_test_csv[client_id])
        trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valloader = DataLoader(val_dataset, batch_size=64)

        return FlowerClient(client_id, net, trainloader, valloader)

    # --- 3. Define the server-side evaluation function ---
    def get_evaluate_fn(test_data_splits):
        """Return an evaluation function for server-side evaluation."""
        def evaluate(server_round: int, parameters: fl.common.NDArrays, config: dict):
            net = ModelCSVIMG(num_csv_features=test_data_splits[0][0].shape[1], img_shape1=32, img_shape2=32).to(DEVICE)

            # Use the last client's test data for server-side evaluation
            server_test_cid = TOTAL_CLIENTS - 1
            params_dict = zip(net.state_dict().keys(), parameters)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            net.load_state_dict(state_dict, strict=True)

            test_dataset = CustomDatasetRes(test_data_splits[0][server_test_cid], test_data_splits[1][server_test_cid],
                                         test_data_splits[2][server_test_cid], test_data_splits[3][server_test_cid])
            testloader = DataLoader(test_dataset, batch_size=64)
            loss, accuracy = test(net, testloader)
            print(f"Server-side evaluation round {server_round} - Loss: {loss:.4f} | Accuracy: {accuracy:.4f}")
            return loss, {"accuracy": accuracy}
        return evaluate

    # --- 4. Define the strategy ---
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=0.5,  # Train on 50% of clients (6 out of 12)
        min_fit_clients=6,
        fraction_evaluate=0.5, # Evaluate on 50% of clients
        min_evaluate_clients=6,
        min_available_clients=TOTAL_CLIENTS,
        evaluate_fn=get_evaluate_fn((X_test_csv, X_test_1, X_test_2, Y_test_csv)), # Server-side evaluation
    )

    # --- 5. Start the simulation ---
    print("Starting Flower simulation...")
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=TOTAL_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=10), # Run for 10 rounds
        strategy=strategy,
        client_resources={"num_cpus": 2, "num_gpus": 0.5 if torch.cuda.is_available() else 0},
    )

if __name__ == "__main__":
    main()

Loading data for all clients...


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


Starting Flower simulation...


2025-08-20 03:12:32,954	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:172.30.170.62': 1.0, 'node:__internal_head__': 1.0, 'CPU': 16.0, 'object_store_memory': 5378616115.0, 'memory': 10757232231.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 0.5}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=2275) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2275) 
(ClientAppActor pid=2275)

Server-side evaluation round 0 - Loss: 2.4826 | Accuracy: 0.1522


...

(ClientAppActor pid=2275) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2275) 
(ClientAppActor pid=2275)             This is a deprecated feature. It will be removed
(ClientAppActor pid=2275)             entirely in future versions of Flower.
(ClientAppActor pid=2275)         
Training:  87%|████████▋ | 156/180 [00:01<00:00, 129.38it/s] [repeated 36x across cluster]
(ClientAppActor pid=2275) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
Testing: 100%|██████████| 93/93 [00:00<00:00, 188.19it/s]
(ClientAppActor pid=2274) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(co

Server-side evaluation round 9 - Loss: 1.6504 | Accuracy: 0.9685


(ClientAppActor pid=2275) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2275) 
(ClientAppActor pid=2275)             This is a deprecated feature. It will be removed
(ClientAppActor pid=2275)             entirely in future versions of Flower.
(ClientAppActor pid=2275)         
(ClientAppActor pid=2275) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
Testing: 100%|██████████| 95/95 [00:00<00:00, 287.91it/s]
(ClientAppActor pid=2274) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. Y

Server-side evaluation round 10 - Loss: 1.6510 | Accuracy: 0.9674


(ClientAppActor pid=2274) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2274) 
(ClientAppActor pid=2274)             This is a deprecated feature. It will be removed
(ClientAppActor pid=2274)             entirely in future versions of Flower.
(ClientAppActor pid=2274)         
(ClientAppActor pid=2274) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
Testing: 100%|██████████| 93/93 [00:00<00:00, 285.43it/s]
(ClientAppActor pid=2275) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. Y